In [130]:
!pip install xgboost catboost lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00


In [158]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, make_scorer, root_mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
import joblib

In [62]:
data_train = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/test.csv')

full = pd.concat([data_train, data_test], sort=False)
full

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,NaN


# EDA

## Аналитика

In [63]:
full.isna().sum()[full.isna().sum() > 0].sort_values()

,0
Exterior2nd,1
Exterior1st,1
BsmtUnfSF,1
BsmtFinSF2,1
BsmtFinSF1,1
KitchenQual,1
Electrical,1
TotalBsmtSF,1
GarageArea,1
GarageCars,1


In [64]:
full.select_dtypes(include=['object']).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [65]:
for col in full.select_dtypes(include=['object']).columns:
    print(f"Столбец '{col}':")
    print(full[col].unique())
    print("-" * 40)

Столбец 'MSZoning':
['RL' 'RM' 'C (all)' 'FV' 'RH' nan]
----------------------------------------
Столбец 'Street':
['Pave' 'Grvl']
----------------------------------------
Столбец 'Alley':
[nan 'Grvl' 'Pave']
----------------------------------------
Столбец 'LotShape':
['Reg' 'IR1' 'IR2' 'IR3']
----------------------------------------
Столбец 'LandContour':
['Lvl' 'Bnk' 'Low' 'HLS']
----------------------------------------
Столбец 'Utilities':
['AllPub' 'NoSeWa' nan]
----------------------------------------
Столбец 'LotConfig':
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
----------------------------------------
Столбец 'LandSlope':
['Gtl' 'Mod' 'Sev']
----------------------------------------
Столбец 'Neighborhood':
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
-------------------------------

In [66]:
full.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,1460.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,180921.195890
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129975.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


## FE

### Преобразование типов данных

In [67]:
full = pd.get_dummies(full, columns=[
    'MSZoning', 'Street', 'Alley', 'LandContour', 'Utilities', 'LotConfig',
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
    'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
    'Foundation', 'Heating', 'CentralAir', 'Electrical', 'FireplaceQu',
    'GarageType', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
])

quality_map = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}

for col in ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond']:
    full[col] = full[col].map(quality_map)

# BsmtExposure
bsmt_exposure_map = {'No': 0, 'Mn': 1, 'Av': 2, 'Gd': 3}
full['BsmtExposure'] = full['BsmtExposure'].map(bsmt_exposure_map)

# Functional
functional_map = {'Typ': 0, 'Min1': 1, 'Min2': 2, 'Mod': 3, 'Maj1': 4, 'Maj2': 5, 'Sev': 6}
full['Functional'] = full['Functional'].map(functional_map)

# GarageFinish
garage_finish_map = {'Unf': 0, 'RFn': 1, 'Fin': 2}
full['GarageFinish'] = full['GarageFinish'].map(garage_finish_map)

In [68]:
# LotShape
lotshape_map = {'IR3': 0, 'IR2': 1, 'IR1': 2, 'Reg': 3}
full['LotShape'] = full['LotShape'].map(lotshape_map)

# LandSlope
landslope_map = {'Gtl': 0, 'Mod': 1, 'Sev': 2}
full['LandSlope'] = full['LandSlope'].map(landslope_map)

# BsmtFinType1 и BsmtFinType2
bsmtfin_map = {'Unf': 0, 'LwQ': 1, 'Rec': 2, 'BLQ': 3, 'ALQ': 4, 'GLQ': 5}

full['BsmtFinType1'] = full['BsmtFinType1'].fillna('Unf').map(bsmtfin_map)
full['BsmtFinType2'] = full['BsmtFinType2'].fillna('Unf').map(bsmtfin_map)

### Заполнение пропусков

In [69]:
# Список категориальных признаков из твоего запроса
cat_cols = ['KitchenQual', 'Functional', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'GarageFinish', 'GarageQual', 'GarageCond']

# Маппинги для Ordinal Encoding + добавляем 'Unknown' как -1
ordinal_maps = {
    'KitchenQual': {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4},
    'Functional': {'Sal': 0, 'Sev': 1, 'Maj2': 2, 'Maj1': 3, 'Mod': 4, 'Min2': 5, 'Min1': 6, 'Typ': 7},
    'BsmtQual': {'None': 0, 'No': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtCond': {'None': 0, 'No': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtExposure': {'None': 0, 'No': 0, 'Mn': 1, 'Av': 2, 'Gd': 3},
    'GarageFinish': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
    'GarageQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
}

for col in cat_cols:
    if col in full.columns:
        # Заполняем пропуски строкой "None" или "Unknown"
        full[col] = full[col].fillna('None')

        # Применяем маппинг, несуществующие значения получают -1
        full[col] = full[col].map(ordinal_maps[col]).fillna(-1).astype(int)

# Числовые признаки подвала и гаража — заполняем пропуски нулём
numeric_zero_fill = [
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    'BsmtFullBath', 'BsmtHalfBath',
    'GarageCars', 'GarageArea',
    'GarageYrBlt'
]
for col in numeric_zero_fill:
    if col in full.columns:
        full[col] = full[col].fillna(0)

# MasVnrArea — заполняем 0
if 'MasVnrArea' in full.columns:
    full['MasVnrArea'] = full['MasVnrArea'].fillna(0)

# LotFrontage — заполняем медианой
if 'LotFrontage' in full.columns:
    full['LotFrontage'] = full['LotFrontage'].fillna(full['LotFrontage'].median())

# ML

In [96]:
train_data = full[full['SalePrice'].notna()]
test_data = full[full['SalePrice'].isna()]

In [97]:
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('SalePrice', axis=1), train_data['SalePrice'], test_size=0.2, random_state=42)

## XGB

In [142]:
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.5],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1],
    'min_child_weight': [1, 3, 5]
}


xgb = XGBRegressor(
    random_state=42
)

random_xgb = RandomizedSearchCV(
    n_iter=30,
    estimator=xgb,
    param_distributions=param_distributions,
    cv=5,
    verbose=1,
    n_jobs=-1,
    error_score='raise'
)

# Обучение
random_xgb.fit(X_train, y_train)

best_parameters_xgb = random_xgb.best_params_
best_r2_xgb = random_xgb.best_score_

xgb = random_xgb.best_estimator_

rmse_xgb_train = root_mean_squared_error(y_train, xgb.predict(X_train))
rmse_xgb_test = root_mean_squared_error(y_test, xgb.predict(X_test))

# Лучшие параметры
print("Best parameters:", best_parameters_xgb)
print("Best r2 score:", best_r2_xgb)
print("RMSE score on train:", rmse_xgb_train)
print("RMSE score on test:", rmse_xgb_test)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters: {'subsample': 0.8, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.1, 'colsample_bytree': 0.8}
Best r2 score: 0.8619441099779864
RMSE score on train: 14554.308142697404
RMSE score on test: 26541.512278912975


In [156]:
xgb.save_model('xgb_model.json')

## LGB

In [141]:
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1],
    'min_child_weight': [1, 3, 5]
}


lgb = LGBMRegressor(
    random_state=42,
    verbose=-1
)

random_lgb = RandomizedSearchCV(
    n_iter=30,
    estimator=lgb,
    param_distributions=param_distributions,
    cv=5,
    verbose=1,
    n_jobs=-1,
)


# Обучение
random_lgb.fit(X_train, y_train)

best_parameters_lgb = random_lgb.best_params_
best_r2_lgb = random_lgb.best_score_

lgb = random_lgb.best_estimator_

rmse_lgb_train = root_mean_squared_error(y_train, lgb.predict(X_train))
rmse_lgb_test = root_mean_squared_error(y_test, lgb.predict(X_test))

# Лучшие параметры
print("Best parameters:", best_parameters_lgb)
print("Best r2 score:", best_r2_lgb)
print("RMSE score on train:", rmse_lgb_train)
print("RMSE score on test:", rmse_lgb_test)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8}
Best r2 score: 0.8643230511633412
RMSE score on train: 14671.777132915044
RMSE score on test: 29622.393835837993


In [159]:
joblib.dump(lgb, 'lgb_model.pkl')

['lgb_model.pkl']

## CatBoost

In [149]:
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'reg_lambda': [0, 0.1, 1],
}


cb = CatBoostRegressor(
    random_state=42,
    verbose=0
)

random_cb = RandomizedSearchCV(
    n_iter=30,
    estimator=cb,
    param_distributions=param_distributions,
    cv=5,
    verbose=1,
    n_jobs=-1,
    error_score='raise'
)

# Обучение
random_cb.fit(X_train, y_train)

best_parameters_cb = random_cb.best_params_
best_r2_cb = random_cb.best_score_

cb = random_cb.best_estimator_

rmse_cb_train = root_mean_squared_error(y_train, cb.predict(X_train))
rmse_cb_test = root_mean_squared_error(y_test, cb.predict(X_test))

# Лучшие параметры
print("Best parameters:", best_parameters_cb)
print("Best r2 score:", best_r2_cb)
print("RMSE score on train:", rmse_cb_train)
print("RMSE score on test:", rmse_cb_test)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters: {'subsample': 0.8, 'reg_lambda': 1, 'n_estimators': 300, 'max_depth': 7, 'learning_rate': 0.1}
Best r2 score: 0.8805687734180154
RMSE score on train: 4417.861114953202
RMSE score on test: 28284.283084987495


In [160]:
joblib.dump(cb, 'cb_model.pkl')

['cb_model.pkl']

# Итоги

In [155]:
submissions = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': xgb.predict(test_data.drop('SalePrice', axis=1))
})
submissions.to_csv('submission.csv', index=False)

In [150]:
results_df = pd.DataFrame({
    'Model': ['XGBoost', 'LightGBM', 'CatBoost'],
    'Best R2': [best_r2_xgb, best_r2_lgb, best_r2_cb],
    'RMSE Train': [rmse_xgb_train, rmse_lgb_train, rmse_cb_train],
    'RMSE Test': [rmse_xgb_test, rmse_lgb_test, rmse_cb_test],
    'Parameters': [best_parameters_xgb, best_parameters_lgb, best_parameters_cb]
})

results_df.to_csv('results.csv')